# Check embeddings available for all years from 2000

In [ ]:
# Reload modules every time before executing the Python code typed
%load_ext autoreload
%autoreload 2

# Import from vm notebook dir
import sys; sys.path.insert(0, '../../')

import os
from datetime import datetime

from ExplainingDriftTextEmbeddings.access.file_storage import FileStorage
from ExplainingDriftTextEmbeddings.access.amazon_pickle_reader import AmazonPickleReader
from ExplainingDriftTextEmbeddings.access.interim_storage import InterimStorage

from BenchTest.amore.amazon_reviews_reader import AmazonReviewsReader

In [ ]:
file_storage = FileStorage()

In [ ]:
amazon_raw_id = 'amazon_raw.pickle'
print(file_storage.get_filepath(amazon_raw_id))

amazon_directoy = os.path.dirname(file_storage.get_filepath(amazon_raw_id))
print(amazon_directoy)

amazon_reader = AmazonPickleReader(amazon_directoy)

## Get complete ID mapping

In [ ]:
interim_storage = InterimStorage('originalno_to_rawid')
print(interim_storage.get_filepath())

if(not interim_storage.isfile()):

    # Config
    amazon_raw_id = 'amazon_raw.pickle'
    print(file_storage.get_filepath(amazon_raw_id))

    amazon_directoy = os.path.dirname(file_storage.get_filepath(amazon_raw_id))
    print(amazon_directoy)
    
    originalno_to_rawid = amazon_reader.originalno_to_rawid
    interim_storage.write(originalno_to_rawid)
else:
    originalno_to_rawid = interim_storage.read()

print(len(originalno_to_rawid))

## Get original numbers for available embeddings

In [ ]:
number_of_all_reviews = 7911684
first_available_rawid = 84090
print(number_of_all_reviews - first_available_rawid)

orig_numbers_with_emb = {} # faster lookup than list
for item in originalno_to_rawid.items():
    if(item[1] >= first_available_rawid):
        orig_numbers_with_emb[item[0]] = item[0]
print(len(orig_numbers_with_emb))

## Check available embeddings

In [ ]:
i = 0
for item in AmazonReviewsReader(file_storage.get_filepath('amazon_gz_file'), 'fields', max_docs=-1, min_year=2000, max_year=2010):
    #score = int(float(item['score']))
    #year = datetime.fromtimestamp(int(item['time'])).year
    number = item['number']
    if not number in orig_numbers_with_emb.keys():
        print(number)
    elif i % 500000 == 0:
        print('[',i,']')
    i += 1

print(i)

#[ 0 ]
#[ 500000 ]
#[ 1000000 ]
#[ 1500000 ]
#[ 2000000 ]
#[ 2500000 ]
#[ 3000000 ]
#[ 3500000 ]
#[ 4000000 ]
#[ 4500000 ]
#[ 5000000 ]
#[ 5500000 ]
#[ 6000000 ]
# 6184084

In [ ]:
# test
for item in AmazonReviewsReader(file_storage.get_filepath('amazon_gz_file'), 'fields', max_docs=2, min_year=1999, max_year=1999):
    number = item['number']
    if not number in orig_numbers_with_emb:
        print(number)

In [ ]:
print(originalno_to_rawid[6184084])
# 6788845

In [ ]:
print(amazon_reader.get_text(6788845, metadata=True))
print(amazon_reader.get_bow50(6788845, metadata=True))
print(amazon_reader.get_bow50(6788845, metadata=False))

# -> test not working now. Maybe create dataset first, check available embeddings later